# 5. Ensambling de los diferentes algoritmos

###### En este notebook, a partir de los resultados obtenidos por los diferentes algoritmos, haciendo uso de los que mayor porcentaje han tenido se generará ensambling entre estos según el peso indicado.

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

## Inicializacion variables

In [2]:
dfLR = None
dfLSVC = None
dfNB = None
dfSGD = None
dfDT = None

#Deep Learning
dfCNN = None
dfLSTM = None

> Lectura de todos los ficheros a tratar, se requerirá de un archivo CSV con todos los campos igual que un te test para en caso que un df no tenga valores por que sea comentado este será utilizado sin peso alguno en los resultados finales 

In [3]:
path = '../data/processed/ensambling/'

# Se necesita de un archivo csv a modo de ejemplo para modificarlo totalmente, su contenido no será usado para nada.
sample = pd.read_csv(path + 'novalidSample.csv', encoding='utf-8')


#Machine learning
dfLR = pd.read_csv(path + 'lr.csv', encoding='utf-8')
dfLSVC = pd.read_csv(path + 'lsvc.csv', encoding='utf-8')
dfNB = pd.read_csv(path + 'nb.csv', encoding='utf-8')
dfSGD = pd.read_csv(path + 'sgd.csv', encoding='utf-8')
dfDT = pd.read_csv(path + 'dt.csv', encoding='utf-8')

#Deep Learning
dfCNN = pd.read_csv(path + 'cnn.csv', encoding='utf-8')
dfLSTM = pd.read_csv(path + 'lstm.csv', encoding='utf-8')

FileNotFoundError: File b'../data/processed/ensambling/lr.csv' does not exist

In [4]:
columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

## Pesos para los diferentes algoritmos

In [5]:
wLR = 0.50
wLSVC = 0.0
wNB = 0.0
wSGD = 0.0
wDT = 0.0

wCNN = 0.50
wLSTM = 0.0

In [6]:
if dfLR is None:
    dfLR = sample
    wLR = 0.0
if dfLSVC is None:
    dfLSVC = sample
    wLSVC = 0.0
if dfNB is None:
    dfNB = sample
    wNB = 0.0
if dfSGD is None:
    dfSGD = sample
    wSGD = 0.0
if dfDT is None:
    dfDT = sample
    wDT = 0.0
if dfCNN is None:
    dfCNN = sample
    wCNN = 0.0
if dfLSTM is None:
    dfLSTM = sample
    wLSTM = 0.0

wTotal = sum([wLR, wLSTM, wNB, wSGD, wDT, wCNN, wLSTM])
if wTotal != 1.0:
    print("El peso total no es de 1. Es de --> " + str(wTotal))
else:
    print("El peso es correcto")

El peso total no es de 1. Es de --> 0.0


In [7]:
dfEnd = sample.copy()
dfEnd[columns] = (dfLR[columns] * wLR 
                  + dfLSVC[columns] * wLSVC 
                  + dfNB[columns] * wNB
                  + dfSGD[columns] * wSGD
                  + dfDT[columns] * wDT
                  + dfCNN[columns] * wCNN
                  + dfLSTM[columns] * wLSTM
                 )

> Exportación final del resputado

In [8]:
name = "ensambled"
dfEnd.to_csv(path + 'ensambled/' + name + '.csv', index=False)

### Ensambling por correlación de las predicciones

In [9]:
coefDensidad = 0.1
maxCorrelacion = 0.98

# maxCorrelacion y coefDensidad Nunca puede ser superior a 1 o inferior a 0
assert maxCorrelacion >= 0.0 and maxCorrelacion <= 1.0
assert coefDensidad >= 0.0 and coefDensidad <= 1.0


In [11]:
def loadPredictions():
    files = os.listdir(path)
    lstCSVs = []
    for f in files:
        if f.endswith(".csv"):
            lstCSVs.append(f)
    frames = {f:pd.read_csv(path+f).sort_values('id') for f in lstCSVs}
    return frames


def getCorrelationMatrix(col,frames):
    corDf = pd.DataFrame()
    for name, df in frames.items():
        corDf[name] = df[col]
    cor = corDf.corr()
    for name in cor.columns:
        cor.set_value(name,name,0.0)
    return cor


def getMaxCorrelation(matrixCorr):
    nCor = np.array(matrixCorr.values)
    corr = np.max(nCor)
    idx = np.unravel_index(np.argmax(nCor, axis=None), nCor.shape)
    x1 = matrixCorr.columns[idx[0]]
    x2 = matrixCorr.columns[idx[1]]
    return corr,x1,x2


def mergeDensitiesWeights(m1,m2,densities):
    d1 = densities[m1]
    d2 = densities[m2]
    d_tot = d1 + d2
    weights1 = 0.5*coefDensidad + (d1/d_tot)*(1-coefDensidad)
    weights2 = 0.5*coefDensidad + (d2/d_tot)*(1-coefDensidad)
    return weights1, weights2


def ensambleByColumn(col,frames,densities):
    if len(frames) == 1:
        _, fr = frames.popitem()
        return fr[col]

    mat = getCorrelationMatrix(col,frames)
    corr,merge1,merge2 = getMaxCorrelation(mat)
    new_col_name = merge1 + '_' + merge2

    w1,w2 = mergeDensitiesWeights(merge1,merge2,densities)
    new_df = pd.DataFrame()
    new_df[col] = (frames[merge1][col]*w1) + (frames[merge2][col]*w2)
    del frames[merge1]
    del frames[merge2]
    frames[new_col_name] = new_df

    if corr >= maxCorrelacion:
        print('\t',merge1,merge2,' (OVER CORR)')
        densities[new_col_name] = max(densities[merge1],densities[merge2])
    else:
        print('\t',merge1,merge2)
        densities[new_col_name] = densities[merge1] + densities[merge2]

    del densities[merge1]
    del densities[merge2]
    #print(densities)
    return ensambleByColumn(col,frames,densities)


ensambled = sample.sort_values('id')

for col in tqdm(columns):
    frames = loadPredictions()
    print('\n\n',col)
    densities = {k:1.0 for k in frames.keys()}
    ensambled[col] = ensambleByColumn(col,frames,densities)

ensambled.to_csv(path + 'ensambled/ensambledByCorrelations.csv', index=False)


  0%|          | 0/6 [00:00<?, ?it/s]C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)




 toxic
	 DT_BOW.csv ensambled.csv
	 LR_TFID.csv predTestLogistic1000.csv  (OVER CORR)
	 NB_BOW158627.csv novalidSample.csv  (OVER CORR)
	 predTestLinear10.csv predTestLogistic10.csv  (OVER CORR)
	 SGD_TFID.csv SGD_TFID158627.csv  (OVER CORR)
	 predTestNB10.csv predTestLinear10.csv_predTestLogistic10.csv  (OVER CORR)
	 LR_TFID100000.csv LR_TFID50000.csv  (OVER CORR)
	 ensambledTop2.csv predTestLSTM_Seq_100000.csv  (OVER CORR)
	 predTestSGD10.csv predTestNB10.csv_predTestLinear10.csv_predTestLogistic10.csv  (OVER CORR)
	 ensambledTopMRDL.csv LR_TFID100000.csv_LR_TFID50000.csv  (OVER CORR)
	 LR_TFID.csv_predTestLogistic1000.csv ensambledTopMRDL.csv_LR_TFID100000.csv_LR_TFID50000.csv  (OVER CORR)
	 NB_TFID.csv NB_TFID158627.csv  (OVER CORR)
	 LR_TFID158627.csv LR_TFID.csv_predTestLogistic1000.csv_ensambledTopMRDL.csv_LR_TFID100000.csv_LR_TFID50000.csv  (OVER CORR)
	 NB_BOW.csv NB_BOW158627.csv_novalidSample.csv  (OVER CORR)
	 DT_TFID.csv DT_TFID158627.csv
	 ensambledSame.csv LR_TFID15862

	 predTestDecisionTree10.csv_predTestSGD10.csv_predTestNB10.csv_predTestLinear10.csv_predTestLogistic10.csv NB_TFID.csv_NB_TFID158627.csv_DT_BOW.csv_ensambled.csv_DT_TFID.csv_DT_TFID158627.csv_predTestDecisionTree200.csv_predTestNB1000.csv_SGD_TFID.csv_SGD_TFID158627.csv_NB_BOW.csv_NB_BOW158627.csv_novalidSample.csv_LSVC_BOW.csv_LR_BOW.csv_SGD_BOW.csv_ensambleddeepL.csv_predTestCNN_Seq_100000.csv_ensambledTop2.csv_predTestLSTM_Seq_100000.csv_ensambledSame.csv_LR_TFID158627.csv_LR_TFID.csv_predTestLogistic1000.csv_ensambledTopMRDL.csv_LR_TFID100000.csv_LR_TFID50000.csv_LSVC_TFID.csv_LSVC_TFID158627.csv_predTestDecisionTree100.csv_predTestDecisionTree50.csv_predTestNB100.csv_predTestNB50.csv_predTestLinear100.csv_predTestSGD50.csv_predTestLinear50.csv_predTestSGD100.csv_predTestLinear.csv_predTestNB200.csv_predTestSGD1000.csv_predTestLinear200.csv_predTestSGD200.csv
	 predTestCNN200.csv predTestCNN1000.csv_predTestLSTM50.csv
	 predTestDecisionTree10.csv_predTestSGD10.csv_predTestNB10.csv


 17%|█▋        | 1/6 [01:20<06:43, 80.72s/it]



 severe_toxic
	 DT_BOW.csv ensambled.csv
	 DT_TFID.csv predTestSGD10.csv
	 DT_TFID158627.csv predTestSGD100.csv
	 ensambleddeepL.csv predTestSGD1000.csv
	 ensambledSame.csv predTestSGD200.csv
	 ensambledTop2.csv predTestSGD50.csv
	 LR_TFID.csv predTestLogistic1000.csv  (OVER CORR)
	 NB_BOW158627.csv novalidSample.csv  (OVER CORR)
	 predTestLinear10.csv predTestLogistic10.csv  (OVER CORR)
	 SGD_TFID.csv SGD_TFID158627.csv  (OVER CORR)
	 LR_TFID100000.csv LR_TFID50000.csv  (OVER CORR)
	 ensambledTopMRDL.csv LR_TFID100000.csv_LR_TFID50000.csv
	 LR_TFID158627.csv ensambledTopMRDL.csv_LR_TFID100000.csv_LR_TFID50000.csv
	 LR_TFID.csv_predTestLogistic1000.csv LR_TFID158627.csv_ensambledTopMRDL.csv_LR_TFID100000.csv_LR_TFID50000.csv
	 NB_BOW.csv NB_BOW158627.csv_novalidSample.csv
	 NB_TFID.csv NB_TFID158627.csv
	 predTestLSTM_Seq_100000.csv ensambledTop2.csv_predTestSGD50.csv
	 ensambledSame.csv_predTestSGD200.csv LR_TFID.csv_predTestLogistic1000.csv_LR_TFID158627.csv_ensambledTopMRDL.csv_LR

...

	 NB_TFID.csv_NB_TFID158627.csv SGD_BOW.csv_predTestDecisionTree200.csv_predTestDecisionTree100.csv_predTestDecisionTree50.csv_LR_BOW.csv_LSVC_TFID.csv_LSVC_TFID158627.csv_NB_BOW.csv_NB_BOW158627.csv_novalidSample.csv_SGD_TFID.csv_SGD_TFID158627.csv_ensambledSame.csv_LR_TFID.csv_predTestLogistic1000.csv_LR_TFID158627.csv_ensambledTopMRDL.csv_LR_TFID100000.csv_LR_TFID50000.csv_predTestCNN_Seq_100000.csv_ensambleddeepL.csv_ensambledTop2.csv_predTestLSTM_Seq_100000.csv_predTestLinear.csv_predTestNB1000.csv_predTestNB200.csv_DT_BOW.csv_ensambled.csv_DT_TFID.csv_DT_TFID158627.csv_predTestLinear200.csv_predTestSGD1000.csv_predTestSGD200.csv_predTestLinear100.csv_predTestLinear50.csv_predTestSGD100.csv_predTestSGD50.csv_predTestNB100.csv_predTestNB50.csv
	 LSVC_BOW.csv NB_TFID.csv_NB_TFID158627.csv_SGD_BOW.csv_predTestDecisionTree200.csv_predTestDecisionTree100.csv_predTestDecisionTree50.csv_LR_BOW.csv_LSVC_TFID.csv_LSVC_TFID158627.csv_NB_BOW.csv_NB_BOW158627.csv_novalidSample.csv_SGD_TFID.cs


 83%|████████▎ | 5/6 [07:00<01:24, 84.17s/it]



 identity_hate
	 DT_BOW.csv ensambled.csv
	 DT_TFID.csv predTestLinear10.csv
	 DT_TFID158627.csv predTestLogistic10.csv
	 ensambleddeepL.csv predTestNB10.csv
	 ensambledSame.csv predTestSGD10.csv
	 ensambledTop2.csv predTestSGD100.csv
	 ensambledTopMRDL.csv predTestSGD1000.csv
	 LR_BOW.csv predTestSGD200.csv
	 LR_TFID.csv predTestSGD50.csv
	 NB_BOW158627.csv novalidSample.csv  (OVER CORR)
	 predTestLogistic1000.csv LR_TFID.csv_predTestSGD50.csv  (OVER CORR)
	 SGD_TFID.csv SGD_TFID158627.csv  (OVER CORR)
	 LR_TFID100000.csv LR_TFID50000.csv  (OVER CORR)
	 ensambledTopMRDL.csv_predTestSGD1000.csv LR_TFID100000.csv_LR_TFID50000.csv
	 LR_TFID158627.csv ensambledTopMRDL.csv_predTestSGD1000.csv_LR_TFID100000.csv_LR_TFID50000.csv
	 NB_TFID.csv NB_TFID158627.csv
	 predTestLogistic1000.csv_LR_TFID.csv_predTestSGD50.csv LR_TFID158627.csv_ensambledTopMRDL.csv_predTestSGD1000.csv_LR_TFID100000.csv_LR_TFID50000.csv
	 ensambledSame.csv_predTestSGD10.csv predTestLogistic1000.csv_LR_TFID.csv_predTes

	 predTestLinear200.csv_predTestLinear100.csv_predTestLinear50.csv predTestLinear.csv_LSVC_BOW.csv_NB_TFID.csv_NB_TFID158627.csv_predTestLSTM_Seq_100000.csv_predTestCNN_Seq_100000.csv_ensambleddeepL.csv_predTestNB10.csv_NB_BOW.csv_NB_BOW158627.csv_novalidSample.csv_DT_BOW.csv_ensambled.csv_DT_TFID.csv_predTestLinear10.csv_DT_TFID158627.csv_predTestLogistic10.csv_SGD_TFID.csv_SGD_TFID158627.csv_ensambledTop2.csv_predTestSGD100.csv_ensambledSame.csv_predTestSGD10.csv_predTestLogistic1000.csv_LR_TFID.csv_predTestSGD50.csv_LR_TFID158627.csv_ensambledTopMRDL.csv_predTestSGD1000.csv_LR_TFID100000.csv_LR_TFID50000.csv_LSVC_TFID.csv_LSVC_TFID158627.csv_SGD_BOW.csv_LR_BOW.csv_predTestSGD200.csv_predTestNB50.csv_predTestNB1000.csv_predTestNB100.csv_predTestNB200.csv_predTestDecisionTree200.csv_predTestDecisionTree100.csv_predTestDecisionTree50.csv
	 predTestCNN200.csv_predTestCNN1000.csv_predTestLSTM50.csv predTestLinear200.csv_predTestLinear100.csv_predTestLinear50.csv_predTestLinear.csv_LSVC_B


100%|██████████| 6/6 [08:24<00:00, 84.07s/it]
